##### 주행중 차선을 인식하고, 그 결과를 원본과 변형 영상에 각각 오버레이하여 동시 출력 

- 추가기능:
    - 인식된 차선은 좌표 쌍 (x1, y1), (x2, y2)로 표현
    - ‘G’ 키를 누르면 차선 인식 결과를 토글(표시/숨김)
    - ‘S’ 키를 누르면 현재 화면 영상을 저장(다시 누르면 저장 중지)
    - ‘ESC’ 키 누르면 프로그램이 에러 없이 종료

In [25]:
import cv2
import numpy as np
import sys

# Gaussian smoothing parameters
kernel_size = 3

# Canny Edge Detector parameters
low_threshold = 50
high_threshold = 150

# Hough Transform parameters
rho = 2
theta = 1 * np.pi / 180
threshold = 15
min_line_length = 10
max_line_gap = 20

trap_bottom_width = 0.85  # 이미지 너비의 85% 차지
trap_top_width = 0.07  # 이미지 너비의 7% 차지
trap_height = 0.4  # 이미지 높이의 40% 차지

# 차선 위치 저장 (Moving Average 적용)
prev_left_line = None
prev_right_line = None

# 차선 검출 토글 및 저장 플래그
show_lane = True
save_video = False
out = None

def grayscale(src):
    return cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

def canny(src, low_threshold, high_threshold):
    return cv2.Canny(src, low_threshold, high_threshold)

def gaussian_blur(src, kernel_size):
    return cv2.GaussianBlur(src, (kernel_size, kernel_size), 0)

def region_of_interest(src, vertices):
    mask = np.zeros_like(src)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(src, mask)

def filter_colors(src):
    """ 흰색과 노란색 차선만 필터링 """
    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(src, lower_white, upper_white)
    white_image = cv2.bitwise_and(src, src, mask=white_mask)

    hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(src, src, mask=yellow_mask)

    return cv2.addWeighted(white_image, 1., yellow_image, 1., 0.)

def draw_lines(src, lines, color=(0, 255, 0), thickness=5):
    if lines is None:
        return
    for line in lines:
        if len(line) == 1:  # (N, 1, 4) 구조일 경우
            x1, y1, x2, y2 = line[0]  
        elif len(line) == 4:  # (N, 4) 구조일 경우
            x1, y1, x2, y2 = line
        else:
            print(f"Unexpected line format: {line}")  # 디버깅용
            continue
        
        # 기울기 계산 (기울기가 너무 수평 혹은 수직인 경우 제외)
        dx = x2 - x1
        dy = y2 - y1
        angle = np.arctan2(dy, dx)  # 기울기 계산 (라디안)

        # 기울기가 수평(또는 수직)에 가까운 선을 제외
        if abs(angle) > np.pi / 8 and abs(angle) < 3 * np.pi / 4:  # 45도 이상, 135도 이하인 선만 그리기
            cv2.line(src, (x1, y1), (x2, y2), color, thickness)

def hough_lines(src, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(src, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def weighted_img(src, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, src, β, λ)

In [26]:
# 1. 영상 입력 및 초기 설정
cap = cv2.VideoCapture('./data/drive_cam.mp4')
if not cap.isOpened():
    print("Video open failed!")
    sys.exit()

fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    initial_frame = frame.copy()

    # 2. 전처리: 색 필터, 그레이스케일 변환, 블러링, Canny 에지 검출
    filtered_frame = filter_colors(frame)
    gray = grayscale(filtered_frame)
    blur = gaussian_blur(gray, kernel_size)
    edges = canny(blur, low_threshold, high_threshold)

    # 3. ROI 설정
    vertices = np.array([[
        (int(frame_width * 0.05), frame_height),
        (int(frame_width * 0.3), int(frame_height * 0.6)),
        (int(frame_width * 0.65), int(frame_height * 0.6)),
        (int(frame_width * 0.95), frame_height)
    ]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # 4. 허프 변환을 이용한 선 검출
    lines = hough_lines(masked_edges, rho, theta, 
                        threshold, min_line_length, max_line_gap)

    # 5. 차선 그리기
    lane_frame = np.zeros_like(frame)
    if show_lane and lines is not None:
        draw_lines(lane_frame, lines)  # 차선 그리기 


    annotated_frame = weighted_img(lane_frame, initial_frame)

    cv2.imshow('Original Video', initial_frame)
    cv2.imshow('Processed Video', annotated_frame)

    key = cv2.waitKey(delay)
    if key in [ord('g'), ord('G')]:
        show_lane = not show_lane  # 차선 표시 토글
    elif key in [ord('s'), ord('S')]:
        save_video = not save_video
        if save_video:
            out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
        else:
            out.release()
            out = None
    elif key == 27:  # ESC 키
        break

    if save_video and out is not None:
        out.write(annotated_frame)

cap.release()
cv2.destroyAllWindows()
if out is not None:
    out.release()
    

In [23]:
import cv2
import numpy as np

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img):
    imshape = img.shape
    trap_bottom_width = 0.9
    trap_top_width = 0.1
    trap_height = 0.3
    
    vertices = np.array([
        [
            (int((imshape[1] * (1 - trap_bottom_width)) / 2), imshape[0]),
            (int((imshape[1] * (1 - trap_top_width)) / 2), int(imshape[0] - imshape[0] * trap_height)),
            (int(imshape[1] - (imshape[1] * (1 - trap_top_width)) / 2), int(imshape[0] - imshape[0] * trap_height)),
            (int(imshape[1] - (imshape[1] * (1 - trap_bottom_width)) / 2), imshape[0])
        ]
    ], dtype=np.int32)
    
    mask = np.zeros_like(img)
    ignore_mask_color = 255 if len(img.shape) == 2 else (255,) * img.shape[2]
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    return cv2.bitwise_and(img, mask)

def draw_lines(img, lines, thickness=5):
    if lines is None:
        return
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else np.inf
            if abs(slope) < 0.5:
                continue
            color = (0, 0, 255) if abs(slope) > 1 else (255, 0, 0)
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, alpha=0.8, beta=1., gamma=0.):
    return cv2.addWeighted(initial_img, alpha, img, beta, gamma)

def filter_colors(image):
    white_threshold = 200
    lower_white = np.array([white_threshold, white_threshold, white_threshold])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([90, 100, 100])
    upper_yellow = np.array([110, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    return cv2.addWeighted(white_image, 1., yellow_image, 1., 0.)

def process_frame(frame, show_lane):
    filtered = filter_colors(frame)
    gray = grayscale(filtered)
    blur = gaussian_blur(gray, kernel_size=5)
    edges = canny(blur, low_threshold=50, high_threshold=150)
    masked_edges = region_of_interest(edges)
    
    line_image = hough_lines(masked_edges, rho=2, theta=np.pi/180, threshold=15,
                             min_line_len=40, max_line_gap=20) if show_lane else np.zeros_like(frame)
    
    return weighted_img(line_image, frame)

def main():
    cap = cv2.VideoCapture('./data/drive_cam.mp4')
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    show_lane = True
    save_video = False
    out = None
    
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        processed_frame = process_frame(frame, show_lane)
        cv2.imshow('Lane Detection', processed_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key in [ord('g'), ord('G')]:
            show_lane = not show_lane
        elif key in [ord('s'), ord('S')]:
            save_video = not save_video
            if save_video:
                out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
            else:
                out.release()
                out = None
        elif key == 27:
            break
        
        if save_video and out is not None:
            out.write(processed_frame)

    cap.release()
    if out is not None:
        out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()